Installs and Imports

In [2]:
# !pip install -q xarray_subset_grid@git+https://github.com/asascience-open/xarray-subset-grid.git
# !pip install -qU xarray shapely thalassa xugrid s3fs fsspec cf_xarray h5netcdf
import cf_xarray
import xarray as xr
import fsspec
import shapely
import xugrid as xu
import fsspec
import s3fs
import thalassa
import xarray_subset_grid as xrsg

Open Dataset

In [3]:
bucket_name = 'noaa-gestofs-pds'
key = '_para2/stofs_2d_glo.20230819/stofs_2d_glo.t00z.fields.cwl.nc'
url = f"s3://{bucket_name}/{key}"

fs = fsspec.filesystem("s3", anon=True)

ds = xr.open_dataset(
    fs.open(url),
    chunks={},
    drop_variables=["neta", "nvel", "max_nvdll", "max_nvell"],
    engine='h5netcdf'
)

In [21]:
bbox = (-70, 40, -60, 50)

def info(ds):
  print(f'Data Variables: {ds.data_vars}')
  print(f'Coordinates: {ds.coords}')
  print(f'Dimensions: {ds.dims}')
  print(f'Attributes: {ds.attrs}\n')

1. Xarray-subset-grid

In [5]:
%%time

# ads = ds.subset_grid.grid.subset_bbox(ds, bbox)
ads = ds.subset_grid.grid.subset_bbox(ds, bbox)

CPU times: user 14.4 s, sys: 1.97 s, total: 16.3 s
Wall time: 40.7 s


In [27]:
ads

<xarray.Dataset> Size: 547MB
Dimensions:      (time: 186, node: 356230, nele: 686355, nvertex: 3, mesh: 1,
                  nbou: 262, nvel: 12421)
Coordinates:
  * time         (time) datetime64[ns] 1kB 2023-08-18T19:00:00 ... 2023-08-26...
    x            (node) float64 3MB dask.array<chunksize=(79,), meta=np.ndarray>
    y            (node) float64 3MB dask.array<chunksize=(79,), meta=np.ndarray>
Dimensions without coordinates: node, nele, nvertex, mesh, nbou, nvel
Data variables:
    element      (nele, nvertex) int32 8MB dask.array<chunksize=(44090, 1), meta=np.ndarray>
    adcirc_mesh  (mesh) int32 4B dask.array<chunksize=(1,), meta=np.ndarray>
    nvell        (nbou) int32 1kB dask.array<chunksize=(262,), meta=np.ndarray>
    ibtype       (nbou) int32 1kB dask.array<chunksize=(262,), meta=np.ndarray>
    nbvv         (nvel) int32 50kB dask.array<chunksize=(12421,), meta=np.ndarray>
    depth        (node) float64 3MB dask.array<chunksize=(79,), meta=np.ndarray>
    zeta         (time, node) float64 530MB dask.array<chunksize=(1, 79), meta=np.ndarray>
Attributes: (12/49)
    _FillValue:         -99999.0
    model:              ADCIRC
    version:            56.dev
    grid_type:          Triangular
    description:        2023081900 :-6 hr nowcast and +180 hr forecast ! 32 C...
    agrid:              OceanMesh2D
    ...                 ...
    sfea0:              45.0
    cf:                 0.001
    eslm:               -0.2
    cori:               0.0
    ntif:               8
    nbfr:               0

2. Thalassa

In [6]:
%%time

tds = thalassa.normalize(ds)
tds = thalassa.crop(tds, shapely.box(*bbox))

CPU times: user 6.01 s, sys: 617 ms, total: 6.62 s
Wall time: 16.5 s


In [26]:
tds

<xarray.Dataset> Size: 844MB
Dimensions:        (time: 186, node: 355129, face: 24875336,
                    max_no_vertices: 3, mesh: 1, nbou: 262, nvel: 12421,
                    triface: 684160, three: 3)
Coordinates:
  * time           (time) datetime64[ns] 1kB 2023-08-18T19:00:00 ... 2023-08-...
    lon            (node) float64 3MB dask.array<chunksize=(79,), meta=np.ndarray>
    lat            (node) float64 3MB dask.array<chunksize=(79,), meta=np.ndarray>
Dimensions without coordinates: node, face, max_no_vertices, mesh, nbou, nvel,
                                triface, three
Data variables:
    face_nodes     (face, max_no_vertices) int32 299MB dask.array<chunksize=(2763927, 1), meta=np.ndarray>
    adcirc_mesh    (mesh) int32 4B dask.array<chunksize=(1,), meta=np.ndarray>
    nvell          (nbou) int32 1kB dask.array<chunksize=(262,), meta=np.ndarray>
    ibtype         (nbou) int32 1kB dask.array<chunksize=(262,), meta=np.ndarray>
    nbvv           (nvel) int32 50kB dask.array<chunksize=(12421,), meta=np.ndarray>
    depth          (node) float64 3MB dask.array<chunksize=(79,), meta=np.ndarray>
    zeta           (time, node) float64 528MB dask.array<chunksize=(1, 79), meta=np.ndarray>
    triface_nodes  (triface, three) int32 8MB 7 0 12 0 ... 311346 309614 311345
Attributes: (12/49)
    _FillValue:         -99999.0
    model:              ADCIRC
    version:            56.dev
    grid_type:          Triangular
    description:        2023081900 :-6 hr nowcast and +180 hr forecast ! 32 C...
    agrid:              OceanMesh2D
    ...                 ...
    sfea0:              45.0
    cf:                 0.001
    eslm:               -0.2
    cori:               0.0
    ntif:               8
    nbfr:               0

3. Xugrid

In [7]:
uds = xu.UgridDataset(ds)

In [16]:
%%time

# Checking time only for subsetting operation
uds = uds.ugrid.sel(y=slice(bbox[2], bbox[3]), x=slice(bbox[0], bbox[1]))

CPU times: user 17.3 s, sys: 1.04 s, total: 18.3 s
Wall time: 19.9 s


In [25]:
uds

<xarray.Dataset> Size: 19GB
Dimensions:  (time: 186, node: 12785004, nbou: 262, nvel: 12421)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2023-08-18T19:00:00 ... 2023-08-26T12:...
    x        (node) float64 102MB dask.array<chunksize=(511401,), meta=np.ndarray>
    y        (node) float64 102MB dask.array<chunksize=(511401,), meta=np.ndarray>
  * node     (node) int64 102MB 0 1 2 3 ... 12785000 12785001 12785002 12785003
Dimensions without coordinates: nbou, nvel
Data variables:
    nvell    (nbou) int32 1kB dask.array<chunksize=(262,), meta=np.ndarray>
    ibtype   (nbou) int32 1kB dask.array<chunksize=(262,), meta=np.ndarray>
    nbvv     (nvel) int32 50kB dask.array<chunksize=(12421,), meta=np.ndarray>
    depth    (node) float64 102MB dask.array<chunksize=(511401,), meta=np.ndarray>
    zeta     (time, node) float64 19GB dask.array<chunksize=(1, 511401), meta=np.ndarray>
Attributes: (12/49)
    _FillValue:         -99999.0
    model:              ADCIRC
    version:            56.dev
    grid_type:          Triangular
    description:        2023081900 :-6 hr nowcast and +180 hr forecast ! 32 C...
    agrid:              OceanMesh2D
    ...                 ...
    sfea0:              45.0
    cf:                 0.001
    eslm:               -0.2
    cori:               0.0
    ntif:               8
    nbfr:               0